In [ ]:
import os
import re
import pickle
from typing import List, Dict, Any, Tuple
from dataclasses import dataclass

import numpy as np
import pandas as pd
import faiss

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# ------------------------------------------------------------------------------
# CONFIG: Modify to suit your environment/model as needed
# ------------------------------------------------------------------------------
LLAMA_MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(LLAMA_MODEL_NAME)
# Force a pad token to be eos if not already set
tokenizer.pad_token = tokenizer.eos_token

# Load Model for embeddings and for completion
# (No 8-bit quantization here, just half-precision if GPU is available)
embedding_model = AutoModelForCausalLM.from_pretrained(
    LLAMA_MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,
)
completion_model = AutoModelForCausalLM.from_pretrained(
    LLAMA_MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,
)

# ------------------------------------------------------------------------------
# DATA CLASSES
# ------------------------------------------------------------------------------
@dataclass
class Document:
    """
    Simple wrapper for storing the chunk text and any associated metadata.
    """
    content: str
    metadata: Dict[str, Any] = None


# ------------------------------------------------------------------------------
# DOCUMENT PROCESSOR
# ------------------------------------------------------------------------------
class DocumentProcessor:
    """
    Helper class to load and clean files (PDF, TXT, CSV), then chunk them
    into Document objects.
    """

    @staticmethod
    def clean_text(text: str) -> str:
        """
        Removes extra whitespace, special chars, etc.
        """
        text = re.sub(r"\.{2,}", "", text)
        text = re.sub(r"\s*\u2002\s*", " ", text)
        text = re.sub(r"\s+", " ", text)

        lines = []
        for line in text.split("\n"):
            line = line.strip()
            if not line or all(c in ".-" for c in line):
                continue
            if line.startswith("=== Page"):
                lines.append(line)
                continue
            if re.match(r"^\d+[-–]\d+$", line):
                continue
            lines.append(line)

        return "\n".join(lines)

    @staticmethod
    def load_pdf(file_path: str) -> str:
        """
        Extract text from each page of a PDF. (Requires PyPDF2)
        """
        from PyPDF2 import PdfReader

        text = ""
        with open(file_path, "rb") as f:
            reader = PdfReader(f)
            for page_num, page in enumerate(reader.pages, 1):
                page_text = page.extract_text()
                text += f"\n=== Page {page_num} ===\n{page_text}"
        return DocumentProcessor.clean_text(text)

    @staticmethod
    def load_txt(file_path: str) -> str:
        """
        Load raw text from a .txt file.
        """
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
        return DocumentProcessor.clean_text(text)

    @staticmethod
    def load_csv(file_path: str) -> str:
        """
        Convert CSV to a text blob (requires pandas).
        """
        df = pd.read_csv(file_path, dtype=str)  # read everything as string
        text = df.to_string(index=False)
        return DocumentProcessor.clean_text(text)

    @staticmethod
    def chunk_text(
        text: str,
        chunk_size: int = 1000,
        chunk_overlap: int = 200
    ) -> List[Document]:
        """
        Basic chunking logic using a custom 'split' approach or
        you can leverage e.g. langchain's RecursiveCharacterTextSplitter.
        """

        # Simple approach: split by ~chunk_size while overlapping
        # You can make this more advanced with custom split strategies.
        chunks = []
        start = 0
        end = chunk_size

        while start < len(text):
            chunk = text[start:end]
            if not chunk.strip():
                break
            doc = Document(content=chunk.strip())
            chunks.append(doc)

            # Overlap region
            start = end - chunk_overlap
            end = start + chunk_size

            if start < 0:
                start = 0

        return chunks

    @staticmethod
    def load_and_chunk_file(file_path: str) -> List[Document]:
        """
        Wrapper to load a PDF, TXT, or CSV, clean the text, then chunk.
        """
        ext = os.path.splitext(file_path)[1].lower()
        if ext == ".pdf":
            text = DocumentProcessor.load_pdf(file_path)
        elif ext == ".txt":
            text = DocumentProcessor.load_txt(file_path)
        elif ext == ".csv":
            text = DocumentProcessor.load_csv(file_path)
        else:
            raise ValueError(f"Unsupported file format: {ext}")

        documents = DocumentProcessor.chunk_text(text)
        return documents


# ------------------------------------------------------------------------------
# EMBEDDING & COMPLETION HELPERS
# ------------------------------------------------------------------------------
def get_embedding(text: str) -> List[float]:
    """
    Creates a single text embedding using the last hidden state of the
    LLaMA model, normalized to unit length.
    """
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = embedding_model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]  # last hidden layer
        embedding = hidden_states.mean(dim=1).squeeze()
        embedding = embedding / embedding.norm(p=2)
    return embedding.cpu().numpy().tolist()


def get_embeddings_batch(texts: List[str], batch_size: int = 32) -> List[List[float]]:
    """
    Efficiently get embeddings in batches to avoid repeated overhead calls.
    """
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_encodings = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = embedding_model(**batch_encodings, output_hidden_states=True)
            hidden_states = outputs.hidden_states[-1]
            # We do a mean pooling for each item in the batch
            for idx in range(len(batch)):
                emb = hidden_states[idx].mean(dim=0)
                emb = emb / emb.norm(p=2)
                embeddings.append(emb.cpu().numpy().tolist())
    return embeddings


def get_completion(
    prompt: str,
    max_new_tokens: int = 512,
    temperature: float = 0.7,
) -> str:
    """
    Generates text using the LLaMA model with a basic sampling configuration.
    """
    input_data = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024,  # guard for input length
    ).to(device)

    with torch.no_grad():
        output = completion_model.generate(
            **input_data,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)


# ------------------------------------------------------------------------------
# VECTOR STORE (FAISS)
# ------------------------------------------------------------------------------
class VectorStore:
    """
    Stores embeddings of Documents in a FAISS index and allows for
    similarity search. 
    """
    def __init__(self, persist_directory: str = "rag_index"):
        self.index = None
        self.documents: List[Document] = []
        self.persist_directory = persist_directory
        os.makedirs(persist_directory, exist_ok=True)

    def _get_index_path(self) -> str:
        return os.path.join(self.persist_directory, "faiss.index")

    def _get_documents_path(self) -> str:
        return os.path.join(self.persist_directory, "documents.pkl")

    def load_local_index(self) -> bool:
        """
        Attempts to load an existing FAISS index and documents list from disk.
        Returns True if load is successful, else False.
        """
        index_path = self._get_index_path()
        docs_path = self._get_documents_path()
        if os.path.exists(index_path) and os.path.exists(docs_path):
            try:
                self.index = faiss.read_index(index_path)
                with open(docs_path, "rb") as f:
                    self.documents = pickle.load(f)
                print(f"Loaded index with {len(self.documents)} documents.")
                return True
            except Exception as e:
                print("Error loading index:", e)
        return False

    def save_local_index(self):
        """
        Saves the FAISS index and documents list to disk.
        """
        if self.index is None or not self.documents:
            return
        try:
            faiss.write_index(self.index, self._get_index_path())
            with open(self._get_documents_path(), "wb") as f:
                pickle.dump(self.documents, f)
            print(f"Saved index with {len(self.documents)} documents.")
        except Exception as e:
            print("Error saving index:", e)

    def create_index(self, documents: List[Document], force_recreate: bool = False):
        """
        Creates or loads the vector store index. If force_recreate is True,
        a new index is built from the provided documents.
        """
        if not force_recreate and self.load_local_index():
            return

        print("Creating a new FAISS index...")
        self.documents = documents
        contents = [doc.content for doc in documents]
        embeddings = get_embeddings_batch(contents)

        embedding_dim = len(embeddings[0])
        self.index = faiss.IndexFlatL2(embedding_dim)
        self.index.add(np.array(embeddings).astype("float32"))

        self.save_local_index()

    def search(self, query: str, k: int = 3) -> List[Tuple[Document, float]]:
        """
        Searches for the top-k documents relevant to the given query, returning
        a list of (Document, similarity_score).
        """
        if self.index is None:
            raise ValueError("Index not initialized. Call create_index first.")
        
        query_emb = get_embedding(query)
        distances, indices = self.index.search(
            np.array([query_emb]).astype("float32"),
            k
        )

        # Convert L2 distances to a simple similarity scale: similarity = 1 / (1 + distance)
        similarities = 1 / (1 + distances)
        results = []
        for i, idx in enumerate(indices[0]):
            doc = self.documents[idx]
            sim_score = similarities[0][i]
            results.append((doc, sim_score))
        return results


# ------------------------------------------------------------------------------
# HYBRID SEARCHER (TF-IDF + VectorStore)
# ------------------------------------------------------------------------------
class HybridSearcher:
    """
    Example hybrid approach combining a TF-IDF search with a VectorStore search.
    """
    def __init__(self, persist_directory: str = "rag_index"):
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix = None
        self.documents: List[Document] = []
        self.vector_store = VectorStore(persist_directory)

    def create_index(self, documents: List[Document]):
        """
        Creates a TF-IDF index and a VectorStore index over the same Documents.
        """
        self.documents = documents
        self._initialize_tfidf()
        self.vector_store.create_index(documents, force_recreate=True)

    def _initialize_tfidf(self):
        """
        Build the TF-IDF representation of all document content.
        """
        contents = [doc.content for doc in self.documents]
        self.tfidf_matrix = self.vectorizer.fit_transform(contents)

    def search(self, query: str, k: int = 3) -> List[Tuple[Document, float]]:
        """
        Combines vector store results and TF-IDF results, then returns top-k overall.
        """
        # Vector results
        vector_results = self.vector_store.search(query, k)

        # TF-IDF results
        query_vec = self.vectorizer.transform([query])
        keyword_scores = cosine_similarity(query_vec, self.tfidf_matrix)[0]
        keyword_indices = np.argsort(keyword_scores)[-k:][::-1]
        keyword_results = [(self.documents[i], keyword_scores[i]) for i in keyword_indices]

        # Combine
        seen = set()
        combined = []
        for doc, score in (vector_results + keyword_results):
            if doc.content not in seen:
                seen.add(doc.content)
                combined.append((doc, score))

        # Sort by score desc, return top k
        return sorted(combined, key=lambda x: x[1], reverse=True)[:k]


# ------------------------------------------------------------------------------
# DOCUMENT GENERATION WITH GUIDELINES
# ------------------------------------------------------------------------------
def generate_document_with_guidelines(
    user_query: str,
    guidelines: List[str],
    vector_store: VectorStore,
    k: int = 3,
    max_new_tokens: int = 512,
    temperature: float = 0.7
) -> str:
    """
    Searches the VectorStore for relevant information based on the user_query,
    then iterates over each guideline to generate segments of a final document.

    This version includes metadata from each Document in the context block 
    for more precise references.

    Returns a single combined result string.
    """
    # 1) Pull top-k results from the vector store
    search_results = vector_store.search(user_query, k=k)

    # 2) Build a consolidated context from the top results
    combined_context_parts = []
    for doc, sim_score in search_results:
        metadata_str = ""
        if doc.metadata:
            meta_strings = [f"{key}: {val}" for key, val in doc.metadata.items()]
            metadata_str = "\n".join(meta_strings)

        context_str = (
            f"---\nContent:\n{doc.content}\n"
            f"Metadata:\n{metadata_str}\n"
            f"Similarity Score: {sim_score:.4f}\n---"
        )
        combined_context_parts.append(context_str)

    combined_context = "\n\n".join(combined_context_parts)

    # 3) Loop over each guideline, create a "segment" for each
    final_segments = []
    for idx, guideline in enumerate(guidelines, start=1):
        prompt = f"""
You have the following user query:
{user_query}

Context from relevant documents (with metadata):
{combined_context}

Guideline #{idx}: {guideline}

Based on the user query and the above context, create a concise 
section of a final document that follows this guideline. 
Focus on using the available context effectively.
"""
        segment_response = get_completion(
            prompt,
            max_new_tokens=max_new_tokens,
            temperature=temperature
        )
        formatted_segment = f"### GUIDELINE #{idx}: {guideline}\n{segment_response.strip()}\n"
        final_segments.append(formatted_segment)

    # 4) Combine all segments into one final "document"
    final_document = "\n\n".join(final_segments)
    return final_document


# ------------------------------------------------------------------------------
# USAGE EXAMPLE (Comment out or remove if not needed)
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    # 1) Load and chunk a sample file (PDF/TXT/CSV)
    #    For demonstration, replace 'sample.pdf' with your own file path
    #    or any .txt, .csv file. If PDF, make sure to install PyPDF2.
    file_path = "sample.pdf"  # Change to your file path
    doc_chunks = DocumentProcessor.load_and_chunk_file(file_path)

    # 2) Create a VectorStore and index these chunks
    vs = VectorStore(persist_directory="rag_index")
    vs.create_index(doc_chunks, force_recreate=True)

    # 3) Example guidelines
    my_guidelines = [
        "Provide a step-by-step approach.",
        "Use non-technical language as much as possible.",
        "Incorporate any numerical data precisely from the context."
    ]

    # 4) Generate a combined document addressing each guideline
    user_query = "How do I perform financial forecasting for a startup?"
    final_doc = generate_document_with_guidelines(
        user_query=user_query,
        guidelines=my_guidelines,
        vector_store=vs,
        k=3,
        max_new_tokens=512,
        temperature=0.7
    )

    print("\n===== FINAL DOCUMENT =====\n")
    print(final_doc)
